<a href="https://colab.research.google.com/github/scadusseau/Implementing-a-Q-learning-algorithm/blob/master/RL_tutorial_mountainCar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Q-Learning / Table -> https://pythonprogramming.net/q-learning-reinforcement-learning-python-tutorial/

In [1]:
#Here we install dependencies in order to display the final video on google colab.
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get install x11-utils > /dev/null 2>&1

!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[all] > /dev/null 2>&1

     |████████████████████████████████| 583kB 2.8MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: setuptools 41.6.0
    Uninstalling setuptools-41.6.0:
      Successfully uninstalled setuptools-41.6.0


In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only

import matplotlib.pyplot as plt

import numpy as np

import glob
import io
import base64
from IPython import display as ipythondisplay
from IPython.display import HTML

from pyvirtualdisplay import Display

In [0]:
#In order to display the video.
display = Display(visible=0, size=(800, 600))
display.start()

"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [5]:
env = wrap_env(gym.make("MountainCar-v0"))

LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODES = 10000

SHOW_EVERY = 1000

DISCRETE_OS_SIZE = [20]* len(env.observation_space.high)
discrete_os_win_size = (env.observation_space.high - env.observation_space.low)/DISCRETE_OS_SIZE

def get_discrete_state(state):
  discrete_state = (state - env.observation_space.low) / discrete_os_win_size
  return tuple(discrete_state.astype(np.int))

q_table = np.random.uniform(low=-2, high=0, size=(DISCRETE_OS_SIZE + [env.action_space.n]))#The table is negative because the rewards are globally negative
                                                  # 20 * 20 * 3  possibilities multiplied by the actions (here we have 3 possible actions)

discrete_state = get_discrete_state(env.reset())
print(str(discrete_state) + " C'est la position dans la q table -> " + str(q_table[discrete_state]))

for episodes in range(EPISODES):
  if episodes%SHOW_EVERY == 0:
    render = True
  else:
    render = False
  
  discrete_state = get_discrete_state(env.reset())
  
  done = False
  while not done:
    action = np.argmax(q_table[discrete_state])
    new_state, reward, done, _ = env.step(action)

    new_discrete_state = get_discrete_state(new_state)
    
    if render:
      env.render()
      
    if not done:
      max_future_q = np.max(q_table[new_discrete_state])
      current_q = q_table[discrete_state + (action,)]

      new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)
      q_table[discrete_state + (action,)] = new_q
    elif new_state[0] >= 0.5:
      q_table[discrete_state + (action,)] = 0

    discrete_state = new_discrete_state

env.close()
show_video()

(7, 10) C'est la position dans la q table -> [-1.43098255 -1.28936395 -0.00859008]
